# Homework 01

## Q1

In [1]:
import pandas as pd

df = (
    pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet")
)

display(df.head())

print(df.dtypes)

print(df.shape)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object
(2463931, 19)


Answer: 19 columns

## Q2

In [2]:
display(
    df
    .assign(duration=lambda ddf: (ddf["tpep_dropoff_datetime"] - ddf["tpep_pickup_datetime"]).dt.total_seconds() / 60)
    ["duration"]
    .describe()
)

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

Answer:  The standard deviation is 46.45

## Q3

In [3]:
display(
    df
    .assign(duration=lambda ddf: (ddf["tpep_dropoff_datetime"] - ddf["tpep_pickup_datetime"]).dt.total_seconds() / 60)
    .assign(is_not_outlier=lambda ddf:  ((1 <= ddf["duration"]) & (ddf["duration"] <= 60)).astype(int))
    ["is_not_outlier"]
    .mean()
)

0.9827547930522406

Answer: The percentage of values left after dropping the outliers is 98%

## Q4

In [4]:
from sklearn.feature_extraction import DictVectorizer

x_train = (
    df
    .assign(duration=lambda ddf: (ddf["tpep_dropoff_datetime"] - ddf["tpep_pickup_datetime"]).dt.total_seconds() / 60)
    .assign(is_not_outlier=lambda ddf:  ((1 <= ddf["duration"]) & (ddf["duration"] <= 60)).astype(int))
    .loc[lambda ddf: ddf["is_not_outlier"] == 1]    
    [["PULocationID", "DOLocationID"]]
    .astype(str)
    .to_dict(orient='records')
)

transformer = DictVectorizer()

transformer = transformer.fit(x_train)
x_t = transformer.transform(x_train)

print(x_t.shape)

(2421440, 515)


Answer: The resulting matrix has 515 columns

## Q5

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y_train = (
    df
    .assign(duration=lambda ddf: (ddf["tpep_dropoff_datetime"] - ddf["tpep_pickup_datetime"]).dt.total_seconds() / 60)
    .assign(is_not_outlier=lambda ddf:  ((1 <= ddf["duration"]) & (ddf["duration"] <= 60)).astype(int))
    .loc[lambda ddf: ddf["is_not_outlier"] == 1]
    ["duration"]
    .to_numpy()
)

lr = LinearRegression()
lr_fitted = lr.fit(x_t, y_train)

print(mean_squared_error(y_pred=lr_fitted.predict(x_t), y_true=y_train, squared=False))

6.986190739643801


Answer: The RMSE of the model on the train data is 6.99

## Q6

In [6]:
x_val, y_val = (
    pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet")
    .assign(duration=lambda ddf: (ddf["tpep_dropoff_datetime"] - ddf["tpep_pickup_datetime"]).dt.total_seconds() / 60)
    .assign(is_not_outlier=lambda ddf:  ((1 <= ddf["duration"]) & (ddf["duration"] <= 60)).astype(int))
    .loc[lambda ddf: ddf["is_not_outlier"] == 1]
    .pipe(lambda ddf: (
        ddf
        [["PULocationID", "DOLocationID"]]
        .astype(str)
        .to_dict(orient='records'),
        ddf
        ["duration"]
        .to_numpy()))
)

x_val_t = transformer.transform(x_val)

print(mean_squared_error(y_pred=lr_fitted.predict(x_val_t), y_true=y_val, squared=False))

7.786409671887426


Answer: The RMSE on the validation dataset is 7.79